In [70]:
import json
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, Model, optimizers, losses, metrics, Sequential
JSON_PATH = "../data_rnn/imdb_dataset_prepared.json"

In [56]:
# Set random seed for reproducibility
np.random.seed(0)
tf.random.set_seed(0)

In [57]:
# Load data from JSON file
print("Loading data...")
with open(JSON_PATH, "r") as f:
    data = json.load(f)
print("Data loaded successfully.")


Loading data...
Data loaded successfully.


In [120]:
X_train = np.array(data["X_train"], dtype=np.int32)
y_train = np.array(data["y_train"], dtype=np.float32)
X_test  = np.array(data["X_test"],  dtype=np.int32)
y_test  = np.array(data["y_test"],  dtype=np.float32)
embeddings = np.array(data["embeddings"], dtype=np.float32)
vocab = np.array(data["vocab"])

X_train -= 1
X_test -= 1

In [130]:
print("After shift → max index in X_train:", np.max(X_train), "min:", np.min(X_train))
print("After shift → max index in X_test: ", np.max(X_test),  "min:", np.min(X_test))

After shift → max index in X_train: 12848 min: 0
After shift → max index in X_test:  12848 min: 0


In [124]:
embedding_dim = embeddings.shape[1]
vocab_size = embeddings.shape[0]
sequence_length = X_train.shape[1]
batch_size = 128

In [125]:
print(f"X_train: {X_train.shape}")
print(f"y_train: {y_train.shape}")
print(f"X_test: {X_test.shape}")
print(f"y_test: {y_test.shape}")
print(f"embeddings: {embeddings.shape}")
print(f"vocab: {vocab.shape}")

X_train: (40000, 130)
y_train: (40000,)
X_test: (10000, 130)
y_test: (10000,)
embeddings: (12849, 50)
vocab: (12849,)


In [126]:
# Build model using Sequential API (simpler approach)
model = Sequential([
    layers.Embedding(
        input_dim=vocab_size,
        output_dim=embedding_dim,
        weights=[embeddings],
        trainable=True
    ),
    layers.SimpleRNN(
        units=16,
        activation="relu",
        return_sequences=False
    ),
    layers.Dense(1, activation="sigmoid")
])

In [128]:
# Force the model (and therefore each layer) to create its weight tensors:
model.build(input_shape=(batch_size, sequence_length))

model.compile(
    optimizer=optimizers.RMSprop(),
    loss=losses.BinaryCrossentropy(),
    metrics=['accuracy']
)

In [129]:
# Training parameters
epochs = 12

print("\nStarting training...")

# Simple training with model.fit()
history = model.fit(
    X_train, y_train,
    batch_size=batch_size,
    epochs=epochs,
    validation_data=(X_test, y_test),
    shuffle=True,
    verbose=1  # This will print progress for each epoch
)

print("\nTraining completed!")


Starting training...
Epoch 1/12
313/313 ━━━━━━━━━━━━━━━━━━━━ 16s 45ms/step - accuracy: 0.5098 - loss: 0.6964 - val_accuracy: 0.5039 - val_loss: 0.6936
Epoch 2/12
313/313 ━━━━━━━━━━━━━━━━━━━━ 13s 40ms/step - accuracy: 0.5165 - loss: 0.6915 - val_accuracy: 0.5084 - val_loss: 0.6921
Epoch 3/12
313/313 ━━━━━━━━━━━━━━━━━━━━ 14s 43ms/step - accuracy: 0.5246 - loss: 0.6884 - val_accuracy: 0.5307 - val_loss: 0.6832
Epoch 4/12
313/313 ━━━━━━━━━━━━━━━━━━━━ 13s 42ms/step - accuracy: 0.5770 - loss: 0.6701 - val_accuracy: 0.6952 - val_loss: 0.5899
Epoch 5/12
313/313 ━━━━━━━━━━━━━━━━━━━━ 13s 41ms/step - accuracy: 0.7099 - loss: 0.5876 - val_accuracy: 0.6928 - val_loss: 0.6073
Epoch 6/12
313/313 ━━━━━━━━━━━━━━━━━━━━ 13s 40ms/step - accuracy: 0.7390 - loss: 0.5590 - val_accuracy: 0.7487 - val_loss: 0.5152
Epoch 7/12
313/313 ━━━━━━━━━━━━━━━━━━━━ 13s 41ms/step - accuracy: 0.7530 - loss: 0.5310 - val_accuracy: 0.7746 - val_loss: 0.4934
Epoch 8/12
313/313 ━━━━━━━━━━━━━━━━━━━━ 13s 40ms/step - accuracy: 0.

In [131]:
loss_fn = tf.keras.losses.BinaryCrossentropy()
optimizer = tf.keras.optimizers.RMSprop()

train_loss_metric = tf.keras.metrics.Mean(name="train_loss")
train_acc_metric  = tf.keras.metrics.BinaryAccuracy(name="train_accuracy")
test_loss_metric  = tf.keras.metrics.Mean(name="test_loss")
test_acc_metric   = tf.keras.metrics.BinaryAccuracy(name="test_accuracy")

batch_size = 128
train_dataset = (
    tf.data.Dataset.from_tensor_slices((X_train, y_train))
    .shuffle(buffer_size=10_000, seed=0)
    .batch(batch_size)
)
test_dataset = (
    tf.data.Dataset.from_tensor_slices((X_test, y_test))
    .batch(batch_size)
)

# 5) Training loop
epochs = 12
for epoch in range(1, epochs + 1):
    start_time = time.time()

    # Reset metrics at the start of each epoch
    train_loss_metric.reset_state()
    train_acc_metric.reset_state()
    test_loss_metric.reset_state()
    test_acc_metric.reset_state()

    # --- Training ---
    for batch_x, batch_y in train_dataset:
        with tf.GradientTape() as tape:
            logits = model(batch_x, training=True)
            loss_value = loss_fn(batch_y, logits)
        grads = tape.gradient(loss_value, model.trainable_variables)
        optimizer.apply_gradients(zip(grads, model.trainable_variables))
        train_loss_metric.update_state(loss_value)
        train_acc_metric.update_state(batch_y, logits)

    # --- Validation ---
    for batch_x, batch_y in test_dataset:
        val_logits = model(batch_x, training=False)
        val_loss = loss_fn(batch_y, val_logits)
        test_loss_metric.update_state(val_loss)
        test_acc_metric.update_state(batch_y, val_logits)

    epoch_time = time.time() - start_time
    train_loss = train_loss_metric.result().numpy()
    train_acc  = train_acc_metric.result().numpy()
    test_loss  = test_loss_metric.result().numpy()
    test_acc   = test_acc_metric.result().numpy()

    print(
        f"Epoch: {epoch} ({epoch_time:.2f}s)\t"
        f"Train: (l: {train_loss:.2f}, a: {train_acc:.2f})\t"
        f"Test:  (l: {test_loss:.2f}, a: {test_acc:.2f})"
    )

Epoch: 1 (151.46s)	Train: (l: 0.46, a: 0.82)	Test:  (l: 0.46, a: 0.81)


2025-06-01 13:56:07.074152: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


Epoch: 2 (135.90s)	Train: (l: 0.44, a: 0.83)	Test:  (l: 0.45, a: 0.82)
Epoch: 3 (140.02s)	Train: (l: 0.41, a: 0.84)	Test:  (l: 0.44, a: 0.82)
Epoch: 4 (131.73s)	Train: (l: 0.41, a: 0.84)	Test:  (l: 0.43, a: 0.83)
Epoch: 5 (130.65s)	Train: (l: 0.40, a: 0.85)	Test:  (l: 0.42, a: 0.83)
Epoch: 6 (130.86s)	Train: (l: 0.39, a: 0.85)	Test:  (l: 0.58, a: 0.81)
Epoch: 7 (131.22s)	Train: (l: 0.38, a: 0.85)	Test:  (l: 0.44, a: 0.82)
Epoch: 8 (130.54s)	Train: (l: 0.38, a: 0.86)	Test:  (l: 0.41, a: 0.84)
Epoch: 9 (130.65s)	Train: (l: 0.38, a: 0.86)	Test:  (l: 0.40, a: 0.84)
Epoch: 10 (131.00s)	Train: (l: 0.37, a: 0.86)	Test:  (l: 0.49, a: 0.80)
Epoch: 11 (130.72s)	Train: (l: 0.37, a: 0.86)	Test:  (l: 0.51, a: 0.79)
Epoch: 12 (130.76s)	Train: (l: 0.37, a: 0.87)	Test:  (l: 0.40, a: 0.84)
